In [102]:
import nltk
from nltk.corpus import *

In [104]:
import nltk
from nltk.corpus import brown
from nltk import WordNetLemmatizer
from math import log 
wnl=WordNetLemmatizer()

genre=['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies',
'humor', 'learned', 'lore', 'mystery', 'news', 'religion', 'reviews', 'romance',
'science_fiction']
sents=gutenberg.sents(gutenberg.fileids())+brown.sents(categories=genre)
words=gutenberg.words(gutenberg.fileids())+brown.words(categories=genre)

_Fdist = nltk.FreqDist([wnl.lemmatize(w.lower()) for w in words])

_Sents = [[wnl.lemmatize(j.lower()) for j in i] for i in sents]

def p(x):
       return _Fdist[x]/float(len(_Fdist))

def pxy(x,y):
       return (len(list(filter(lambda s :  x in s and y in s ,_Sents)))+1)/ float(len(_Sents) )

def pmi(x,y):
    return pxy(x,y)/p(x)
#       return  log(pxy(x,y)/(p(x)*p(y)),2) 

In [105]:
import numpy as np
import pandas as pd
training=open("../Desktop/training.txt","r")
validation=open("../Desktop/validation.txt","r")
train_list=training.readlines()
val_list=validation.readlines()
columns=["word1","word2","feature","label"]
training_df=pd.DataFrame(columns=columns)
validation_df=pd.DataFrame(columns=columns)
words=[]
for i in range (len(val_list)):
    w=val_list[i].split(",")
    if w[2][-1]=='?':
        w[2]=w[2][:-1]
    validation_df.loc[i]=[w[0],w[1],w[2],w[3][0]] 
    words.append(w[0])
    words.append(w[1])
    words.append(w[2])
for i in range (len(train_list)):
    w=train_list[i].split(",")
    training_df.loc[i]=[w[0],w[1],w[2],w[3][0]] 
    words.append(w[0])
    words.append(w[1])
    words.append(w[2])

In [106]:
def cal(w1,w2):
    if p(w1)==0:
        w1=wnl.lemmatize(w1)
    if p(w2)==0:
        w2=wnl.lemmatize(w2)
    if p(w1)==0 or p(w2)==0:
        return 0
    else:
        return pmi(w1,w2)


In [107]:
def add_pmi(training_df):
    pmi12=[]
    pmi13=[]
    pmi23=[]
    for index,row in training_df.iterrows():
        
        res12=cal(row[0],row[1])
        res13=cal(row[0],row[2])
        res23=cal(row[1],row[2])
        pmi12.append(res12)
        pmi13.append(res13)
        pmi23.append(res23)
        
    training_df['pmi12']=pmi12
    training_df['pmi13']=pmi13
    training_df['pmi23']=pmi23
    

In [108]:

add_pmi(training_df)

In [109]:
from nltk.corpus import wordnet as wn


def count_feature(word1,wf):
    w1=wn.synsets(word1)
    wds=[]
    for i in w1:
        wds.append(i.definition())
    wds=[wnl.lemmatize(j.lower()).split() for j in wds]
    wds=sum(wds,[])
    return wds.count(wnl.lemmatize(wf))+wds.count(wf)
count_feature('pot','lid')

def add_count(df):
    count13=[]
    count23=[]
    for index,row in df.iterrows():
        count13.append(count_feature(row[0],row[2]))
        count23.append(count_feature(row[1],row[2]))
    df['count13']=count13
    df['count23']=count23
add_count(training_df)


In [110]:
training_df

,word1,word2,feature,label,pmi12,pmi13,pmi23,count13,count23
0,coconut,rice,infested,0,0.081975,0.040987,0.009759,0,0
1,ostrich,emu,cloves,0,0.000000,0.081975,0.000000,0,0
2,bathtub,toilet,barrier,0,0.081975,0.081975,0.019518,0,0
3,pan,skillet,lid,1,0.007590,0.015181,0.204937,0,0
4,pumpkin,spinach,snake,0,0.136625,0.136625,0.136625,0,0
5,crow,crane,flies,1,0.006404,0.012809,0.024110,0,0
6,jeans,shirt,bug,0,0.017078,0.017078,0.013438,0,0
7,raisin,cranberry,clasp,0,0.058553,0.058553,0.409874,0,0
8,pillow,bag,shows,0,0.022155,0.011078,0.002907,0,0
9,tie,blouse,jazz,0,0.005060,0.005060,0.204937,0,0


In [111]:
add_pmi(validation_df)

In [120]:
from nltk.corpus import wordnet as wn
word_set=set(words)
word_dic={}
none_set=[]
for word in word_set:
    word_dic[word]=wn.synsets(word)
    if(word_dic[word]==[]):
        none_set.append(word)
none_set=set(none_set)    
word_dic={}
wordvec=open("../Downloads/wordvector/wordvec.txt","r",errors='ignore')
count=0
for line in wordvec:
    w=line.split(" ",1)[0]
    count+=1
#    print (count)
    if w in word_set:
        wv=line.split(" ")
        wv=wv[1:]
        wv[-1]=wv[-1].rstrip('\n')
        wv=[float(i) for i in wv]
        word_dic[w]=wv        
#        print (len(word_dic))

In [121]:
from scipy.spatial.distance import cosine as cos

def addsim(training_df):   
#    l=[]
    cos_sims_1=[]
    cos_sims_2=[]
    cos_sims_3=[]
    cos_sims_4=[]
    for index,row in training_df.iterrows():
        x1=np.array(word_dic[row[0]])
        x2=np.array(word_dic[row[1]])
        y=np.array(word_dic[row[2]])
#        v1, Comps=compute_pc(np.array([x1,x2,y]), 1)
        cos_sims_1.append(cos(x1, x2))
        cos_sims_2.append(cos(x1,y))
        cos_sims_3.append(cos(x2,y))
        cos_sims_4.append(cos(x1,y)/(1e-5+cos(x2,y)))
#        l.append(v1)
#    training_df["v1"]=l
    training_df["cos(w1,w2)"]=cos_sims_1
    training_df["cos(w1,f)"]=cos_sims_2
    training_df["cos(w2,f)"]=cos_sims_3
    training_df["cos(w1,f)/cos(w2,f)"]=cos_sims_4

addsim(training_df)


In [139]:
training_df

,word1,word2,feature,label,pmi12,pmi13,pmi23,count13,count23,"cos(w1,w2)","cos(w1,f)","cos(w2,f)","cos(w1,f)/cos(w2,f)"
0,coconut,rice,infested,0,0.081975,0.040987,0.009759,0,0,0.350304,0.776556,0.788691,0.984601
1,ostrich,emu,cloves,0,0.000000,0.081975,0.000000,0,0,0.447193,0.826901,0.876629,0.943263
2,bathtub,toilet,barrier,0,0.081975,0.081975,0.019518,0,0,0.308351,0.814430,0.734582,1.108683
3,pan,skillet,lid,1,0.007590,0.015181,0.204937,0,0,0.266692,0.401804,0.566524,0.709231
4,pumpkin,spinach,snake,0,0.136625,0.136625,0.136625,0,0,0.413375,0.681086,0.831447,0.819147
5,crow,crane,flies,1,0.006404,0.012809,0.024110,0,0,0.633139,0.538428,0.732142,0.735405
6,jeans,shirt,bug,0,0.017078,0.017078,0.013438,0,0,0.261077,0.882776,0.820472,1.075924
7,raisin,cranberry,clasp,0,0.058553,0.058553,0.409874,0,0,0.404522,0.949384,0.948450,1.000974
8,pillow,bag,shows,0,0.022155,0.011078,0.002907,0,0,0.491972,0.923094,0.873927,1.056248
9,tie,blouse,jazz,0,0.005060,0.005060,0.204937,0,0,0.534282,0.878574,0.894210,0.982503


In [145]:
from sklearn import svm
from sklearn import linear_model
X=[]
Y=[]
for index, row in training_df.iterrows():
        X.append(row[4:])
        Y.append(row[3])

clf=svm.SVC()
#clf.fit(X,Y)

from sklearn import tree
#clf = tree.DecisionTreeClassifier()
#clf = linear_model.LogisticRegression()
clf.fit(X, Y)
#lr.predict_proba(X_test_std[0,:])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [113]:
add_count(validation_df)

validation_df

,word1,word2,feature,label,pmi12,pmi13,pmi23,count13,count23
0,sandwiches,breakfast,lunch,1,0.031529,0.031529,0.004407,0,0
1,surfboard,raft,water,0,0.000000,0.000000,0.051234,0,2
2,banana,cucumber,long,0,0.081975,0.081975,0.063057,0,0
3,lambs,cattle,wool,1,0.004126,0.004126,0.001485,0,0
4,shrimp,spinach,pink,1,0.136625,0.136625,0.136625,0,0
5,bus,taxi,passengers,0,0.018631,0.009315,0.021572,0,1
6,garage,brick,large,1,0.013662,0.027325,0.024110,0,0
7,walkway,alley,walk,0,0.409874,0.409874,0.051234,0,0
8,polyurethane,polyester,material,0,0.000000,0.000000,0.058553,0,0
9,spoon,potato,wood,1,0.010510,0.010510,0.009108,2,0


In [124]:
addsim(validation_df)

In [146]:
predicted=[]
correct=0
total=0
count_1=0
count_1_right=0
count_all_1=0

for index,row in validation_df.iterrows():
    pre=clf.predict([row[4:]])  
    predicted.append(pre)
    if pre=="1":
        count_1+=1
        if pre==row[3]:
            count_1_right+=1
    if pre==row[3]:
        correct+=1
    if row[3]=="1":
        count_all_1+=1
    total+=1
#validation_df["predicted"]=predicted
accuracy=correct/total
print(count_1)
print(count_1_right)
print(count_all_1)
precision=count_1_right/count_1
recall=count_1_right/count_all_1
print ("accuracy:",accuracy)
print ("precision:",precision)
print ("recall:",recall)
print ("F1:",2*precision*recall/(precision+recall))
#validation_df[:100][["word1","word2","feature","label","predicted"]]


1518
843
1364
accuracy: 0.5606171932402645
precision: 0.5553359683794467
recall: 0.6180351906158358
F1: 0.5850104094378904
